A notebook to make p-value images from the results of whole_bain_linear_mdl_fit

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import copy
import glob
import os
from pathlib import Path
import pickle

import imageio
import matplotlib.colors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyqtgraph as pg
import tifffile

from janelia_core.dataprocessing.dataset import ROIDataset
from janelia_core.utils.data_saving import append_ts
from janelia_core.visualization.volume_visualization import make_z_plane_movie
from janelia_core.visualization.custom_color_maps import make_red_green_c_map
from janelia_core.visualization.custom_color_maps import make_red_blue_green_c_map
from janelia_core.visualization.volume_visualization import make_rgb_z_plane_movie
from janelia_core.visualization.image_generation import rgb_3d_max_project
from janelia_core.visualization.volume_visualization import comb_movies
from janelia_core.visualization.volume_visualization import visualize_rgb_max_project

from keller_zlatic_vnc.visualization import gen_coef_p_vl_cmap
from keller_zlatic_vnc.visualization import visualize_coef_p_vl_max_projs


In [3]:
%matplotlib qt

## Parameters go here

In [4]:
ps = {}

# Location of results of whole_brain_linear_mdl_fit
ps['results_folder'] = r'\\dm11\bishoplab\projects\keller_vnc\results\whole_brain_stats'
ps['results_file'] = 'decision_dependence_quiet_ref_2020_08_31_11_16_07_024481.pkl' # Quiet reference

# A string to add to saved file names
ps['save_supp_str'] = 'decision_dep_quiet_ref'

# Specify type of images we generate
ps['gen_coef_movies'] = False
ps['gen_coef_tiffs'] =  False

ps['gen_p_value_movies'] = False
ps['gen_p_value_tiffs'] = False

ps['gen_filtered_coef_movies'] = False
ps['gen_filtered_coef_tiffs'] = False

ps['gen_combined_movies'] =False
ps['gen_combined_tiffs'] = False
ps['gen_combined_projs'] = True

ps['gen_uber_movies'] = True

# Threshold p-values if we are making threshold images - we will make an image for each p-value
ps['thresholds'] = [.05, .01, .001]

# Specify percentiles we use for mapping min and max coef values to colors - value should be between 0 and 100
ps['coef_clim_percs'] = [1, 99]

# Specify fixed limits for coefficients; if provided coef_clim_percs is ignored.  None indicates this parameter is not used
# and limits for coefficients are calculated based on coef_clim_percs. 

ps['coef_lims'] = [-1, 1]

# Specify the type of colormap we use for coefficients
ps['coef_cmap'] = 'gbr' # gr: green to red; gbr: green to blue to red

# Specify lower percentile we use for mapping p-values to colors - should be between 0 and 100
ps['min_p_val_perc'] = 1

# Specify limits when plotting colormap of log p-values (this is for the colormap legend only, it does not affect the range
# of p-values that are shown in the generated maps)
#ps['log_p_vls_cmap_range'] = (0, -4, -.01)

# Specify percentiles we use for mapping min and max values to colors for mean image - values should be between 0 and 100
ps['mean_img_clim_percs'] = [0.1, 99.9]

# Specify where the original datasets are located - we use these for determining the position of the rois
ps['data_loc_file'] = r'A:\projects\keller_vnc\data\experiment_data_locations.xlsx'
ps['dataset_folder'] = 'extracted'
ps['dataset_base_folder'] = r'K:\\SV4'
ps['roi_group'] = 'rois_1_5_5'

# Specify where we save images
ps['save_folder'] = ps['results_folder']




## Load the results

In [5]:
rs_file = Path(ps['results_folder']) / ps['results_file']
with open(rs_file, 'rb') as f:
    rs = pickle.load(f)

In [6]:
test_behs = list(rs['beh_stats'].keys())
n_rois = len(rs['beh_stats'][test_behs[0]]['p_values'])

## Load a dataset

Because the rois are in the same location for each dataset, we can just look at the first dataset to find the position of the rois

In [7]:
# Read in dataset locations
def c_fcn(str):
    return str.replace("'", "")
converters = {0:c_fcn, 1:c_fcn}

data_locs = pd.read_excel(ps['data_loc_file'], header=1, usecols=[1, 2], converters=converters)

# Read in the first dataset
dataset_path = (Path(ps['dataset_base_folder']) / data_locs['Main folder'][0] / data_locs['Subfolder'][0] / 
                    Path(ps['dataset_folder']) / '*.pkl')
dataset_file = glob.glob(str(dataset_path))[0]

with open(dataset_file, 'rb') as f:
    dataset = ROIDataset.from_dict(pickle.load(f))

## Get ROI locations for first dataset

In [8]:
rois = dataset.roi_groups[ps['roi_group']]['rois']
if len(rois) != n_rois:
    raise(RuntimeError('Number of rois in dataset does not match number of rois statistics are calculated for.'))

## Load mean image

In [9]:
mn_img = dataset.stats['mean']

## Define helper functions

In [10]:
def coef_clims(vls, perc):
    if ps['coef_lims'] is not None:
        print('Using fixed coeficient color limits.')
        return ps['coef_lims']
    else:
        small_v = np.percentile(vls, perc[0])
        large_v = np.percentile(vls, perc[1])
        v = np.max([np.abs(small_v), np.abs(large_v)])
        return [-v, v]

def p_vl_clims(vls, perc):
    small_v = np.percentile(vls, perc)
    return [small_v, 0]

## Generate and save images

In [11]:
# Make folder to save results into
image_folder = Path(ps['results_file']).stem # Save images under a folder with the same name as the results

if ps['coef_lims'] is not None:
    extra_str = '_w_fixed_coef_lims'
else:
    extra_str = ''

save_folder_path= Path(ps['save_folder']) / (image_folder + '_cmap_' + ps['coef_cmap'] + extra_str)

os.makedirs(save_folder_path)


In [12]:
# Save the mean image
mn_image_path = save_folder_path / 'mean.tiff'

imageio.mimwrite(mn_image_path, mn_img)

mn_img_min_c_lim = np.percentile(mn_img,  ps['mean_img_clim_percs'][0])
mn_img_max_c_lim = np.percentile(mn_img, ps['mean_img_clim_percs'][1])

make_z_plane_movie(volume=mn_img, save_path= str(save_folder_path /  'mean.mp4'), 
                   cmap='gray', clim=(mn_img_min_c_lim, mn_img_max_c_lim),
                   title = 'Mean Image', cbar_label='$F$')

Bbox([[0.125, 0.19266371971185325], [0.7450000000000001, 0.7973362802881467]])

In [13]:
im_shape = mn_img.shape

n_vars = len(test_behs)

#coef_cmap = make_red_green_c_map(10000)
#coef_cmap = make_red_blue_green_c_map(n=10000)

if ps['coef_cmap']  == 'gr':
    coef_cmap = matplotlib.colors.LinearSegmentedColormap.from_list(name='red_to_green', 
                                                                colors=[(.00,  [1.0, 0.0, 0.0, 1.0]),
                                                                        (.48,  [0.2, 0.0, 0.0, 1.0]),
                                                                        (.50,  [0.0, 0.0, 0.0, 1.0]),
                                                                        (.52,  [0.0, 0.2, 0.0, 1.0]),
                                                                        (1.0,  [0.0, 1.0, 0.0, 1.0])],
                                                                        N=10000)
elif ps['coef_cmap'] == 'gbr':
    coef_cmap = matplotlib.colors.LinearSegmentedColormap.from_list(name='red_to_blue_to_green', 
                                                                colors=[(.00,  [1.0, 0.0, 0.0, 1.0]),
                                                                        (.45,  [0.2, 0.0, 0.0, 1.0]),
                                                                        (.50,  [0.0, 0.0, 1.0, 1.0]),
                                                                        (.55,  [0.0, 0.2, 0.0, 1.0]),
                                                                        (1.0,  [0.0, 1.0, 0.0, 1.0])],
                                                                        N=10000)
else:
    raise(ValueError('Unrecogonized value for ps[coef_cmap]'))

for v_i in range(n_vars):
    var_name = test_behs[v_i]
    
    coefs_image = np.zeros(im_shape, dtype=np.float32) 
    p_vls_image = np.zeros(im_shape, dtype=np.float32) 

    coefs = rs['beh_stats'][var_name]['beta']
    p_vls = rs['beh_stats'][var_name]['p_values']
    log_p_vls = np.log10(p_vls)
    
    for r_i in range(n_rois):
        cur_voxel_inds = rois[r_i].voxel_inds
        
        coefs_image[cur_voxel_inds] = coefs[r_i]
        p_vls_image[cur_voxel_inds] = p_vls[r_i]
                
    if ps['gen_coef_movies'] or ps['gen_coef_tiffs'] or ps['gen_uber_movies']:
        coef_file_name = var_name + '_' + ps['save_supp_str'] + '_coefs'

        if ps['gen_coef_tiffs']:
            tifffile.imwrite(save_folder_path / (coef_file_name + '.tiff'), coefs_image, compress=6)
 
        if ps['gen_coef_movies'] or ps['gen_uber_movies']:
            coef_movie_path = str(save_folder_path / (coef_file_name + '.mp4'))
            coef_movie_ax_pos = make_z_plane_movie(volume=coefs_image, save_path=coef_movie_path, 
                               cmap=coef_cmap, clim=coef_clims(coefs, ps['coef_clim_percs']), 
                               title = var_name, cbar_label='${\Delta F}/{F}$',
                               one_index_z_plane=True)
        
    if ps['gen_p_value_movies'] or ps['gen_p_value_tiffs'] or ps['gen_uber_movies']:
        p_vl_file_name = var_name + '_' + ps['save_supp_str'] + '_p_vls'
        
        if ps['gen_p_value_tiffs']:
            tifffile.imwrite(save_folder_path / (p_vl_file_name + '.tiff'), p_vls_image, compress=6)
       
        if ps['gen_p_value_movies'] or ps['gen_uber_movies']:
            log_p_vls_image = np.log10(p_vls_image)
            log_p_vls_image[p_vls_image == 0] = 0
        
            p_vl_movie_path = str(save_folder_path / (p_vl_file_name + '.mp4'))
            make_z_plane_movie(volume=log_p_vls_image, save_path=p_vl_movie_path, 
                               cmap='magma_r', clim=p_vl_clims(log_p_vls, ps['min_p_val_perc']), 
                               title = var_name, cbar_label='$\log_{10}(p)$',
                               one_index_z_plane=True)
    
    if ps['gen_filtered_coef_movies'] or ps['gen_filtered_coef_tiffs']:
        for th in ps['thresholds']:
            filtered_coef_file_name = var_name + '_' + ps['save_supp_str'] + '_coefs_p_th_' + str(th)
            
            coefs_image_th = copy.deepcopy(coefs_image)
            
            coefs_image_th[p_vls_image > th] = 0
            
            if ps['gen_filtered_coef_tiffs']:
                tifffile.imwrite(save_folder_path / (filtered_coef_file_name + '.tiff'), coefs_image_th, compress=6)
     
            if ps['gen_filtered_coef_movies']:
                ax_pos = make_z_plane_movie(volume=coefs_image_th, save_path=str(save_folder_path / (filtered_coef_file_name + '.mp4')), 
                                   cmap=coef_cmap, clim=coef_clims(coefs, ps['coef_clim_percs']),
                                   title = var_name + '$, p \leq$' + str(th), cbar_label='${\Delta F}/{F}$')
            
    if ps['gen_combined_movies'] or ps['gen_combined_tiffs'] or ps['gen_combined_projs'] or ps['gen_uber_movies']:
        combined_file_name = var_name + '_' + ps['save_supp_str'] + '_combined'
        
        
        
        log_p_vls_image = np.log10(p_vls_image)
        log_p_vls_image[p_vls_image == 0] = 0
        
        # Generate combined color map
        combined_cmap = gen_coef_p_vl_cmap(coef_cmap=coef_cmap, 
                                           positive_clim=coef_clims(coefs, ps['coef_clim_percs'])[1],
                                           plims=p_vl_clims(log_p_vls, ps['min_p_val_perc']))

        # Make RGB volumes 
        combined_vol = combined_cmap[coefs_image, log_p_vls_image]
        
        combined_vol_uint8 = (combined_vol*255).astype(np.uint8)
        
        n_z_planes = coefs_image.shape[0]
        combined_planes = [np.squeeze(combined_vol[z, :,:,:]) for z in range(n_z_planes)]
        
        # Save tiff stacks of RGB volumes
        if ps['gen_combined_tiffs']:
            tifffile.imwrite(save_folder_path / (combined_file_name + '.tiff'), combined_vol_uint8, compress=6)

            # Save colormaps for combined tiffs
            combined_cmap_file = save_folder_path / (combined_file_name + '_cmap.pkl')
            with open(combined_cmap_file, 'wb') as f:
                pickle.dump(combined_cmap.to_dict(), f)

        # Make videos of RGB volumes
        if ps['gen_combined_movies'] or ps['gen_uber_movies']:
            comb_movie_path = str(save_folder_path / (combined_file_name + '.mp4'))
            make_rgb_z_plane_movie(z_imgs=combined_planes, 
                                   save_path=comb_movie_path,
                                   cmap=combined_cmap, 
                                   title=var_name,
                                   cmap_param_vls=(None, np.arange(combined_cmap.param_vl_ranges[1][1], 
                                                                   combined_cmap.param_vl_ranges[1][0], .01)),
                                   cmap_param_strs = ['coef vl ($\Delta F / F$)', '$\log(p)$'],
                                   one_index_z_plane=True, 
                                   ax_position=coef_movie_ax_pos)
                                           
        if ps['gen_combined_projs']:
            
            visualize_coef_p_vl_max_projs(vol=np.moveaxis(combined_vol, 0, 2), dim_m=np.asarray([1, 1, 5]), 
                                          cmap=combined_cmap,
                                          cmap_coef_range=None, cmap_p_vl_range=None, #ps['log_p_vls_cmap_range'],
                                          title=var_name)
            plt.savefig(save_folder_path / (combined_file_name + '.png'), facecolor=(0,0,0))
            plt.close()
                
    if ps['gen_uber_movies']:
        uber_file_name = var_name + '_' + ps['save_supp_str'] + '_coef_p_vls_comb'
        uber_movie_path = save_folder_path / (uber_file_name + '.mp4')
        comb_movies(movie_paths=[coef_movie_path, p_vl_movie_path, comb_movie_path], save_path=uber_movie_path)
        
        if not ps['gen_coef_movies']:
            os.remove(coef_movie_path)
        if not ps['gen_p_value_movies']:
            os.remove(p_vl_movie_path)
        if not ps['gen_combined_movies']:
            os.remove(comb_movie_path)
                
    print('Done with making images for variable: ' + var_name)
        
        
    

Using fixed coeficient color limits.
Using fixed coeficient color limits.


t:   0%|                                                                              | 0/86 [00:00<?, ?it/s, now=None]

Moviepy - Building video \\dm11\bishoplab\projects\keller_vnc\results\whole_brain_stats\decision_dependence_quiet_ref_2020_08_31_11_16_07_024481_cmap_gbr_w_fixed_coef_lims\B_decision_dep_quiet_ref_coef_p_vls_comb.mp4.
Moviepy - Writing video \\dm11\bishoplab\projects\keller_vnc\results\whole_brain_stats\decision_dependence_quiet_ref_2020_08_31_11_16_07_024481_cmap_gbr_w_fixed_coef_lims\B_decision_dep_quiet_ref_coef_p_vls_comb.mp4



Moviepy - Done !
Moviepy - video ready \\dm11\bishoplab\projects\keller_vnc\results\whole_brain_stats\decision_dependence_quiet_ref_2020_08_31_11_16_07_024481_cmap_gbr_w_fixed_coef_lims\B_decision_dep_quiet_ref_coef_p_vls_comb.mp4
Done with making images for variable: B
Using fixed coeficient color limits.
Using fixed coeficient color limits.


t:   0%|                                                                              | 0/86 [00:00<?, ?it/s, now=None]

...

Moviepy - Building video \\dm11\bishoplab\projects\keller_vnc\results\whole_brain_stats\decision_dependence_quiet_ref_2020_08_31_11_16_07_024481_cmap_gbr_w_fixed_coef_lims\P_decision_dep_quiet_ref_coef_p_vls_comb.mp4.
Moviepy - Writing video \\dm11\bishoplab\projects\keller_vnc\results\whole_brain_stats\decision_dependence_quiet_ref_2020_08_31_11_16_07_024481_cmap_gbr_w_fixed_coef_lims\P_decision_dep_quiet_ref_coef_p_vls_comb.mp4



Moviepy - Done !
Moviepy - video ready \\dm11\bishoplab\projects\keller_vnc\results\whole_brain_stats\decision_dependence_quiet_ref_2020_08_31_11_16_07_024481_cmap_gbr_w_fixed_coef_lims\P_decision_dep_quiet_ref_coef_p_vls_comb.mp4
Done with making images for variable: P
Using fixed coeficient color limits.
Using fixed coeficient color limits.


t:   0%|                                                                              | 0/86 [00:00<?, ?it/s, now=None]

Moviepy - Building video \\dm11\bishoplab\projects\keller_vnc\results\whole_brain_stats\decision_dependence_quiet_ref_2020_08_31_11_16_07_024481_cmap_gbr_w_fixed_coef_lims\T_decision_dep_quiet_ref_coef_p_vls_comb.mp4.
Moviepy - Writing video \\dm11\bishoplab\projects\keller_vnc\results\whole_brain_stats\decision_dependence_quiet_ref_2020_08_31_11_16_07_024481_cmap_gbr_w_fixed_coef_lims\T_decision_dep_quiet_ref_coef_p_vls_comb.mp4



Moviepy - Done !
Moviepy - video ready \\dm11\bishoplab\projects\keller_vnc\results\whole_brain_stats\decision_dependence_quiet_ref_2020_08_31_11_16_07_024481_cmap_gbr_w_fixed_coef_lims\T_decision_dep_quiet_ref_coef_p_vls_comb.mp4
Done with making images for variable: T


In [14]:
1

1

In [15]:
rs['ps']

{'data_folder': 'A:\\projects\\keller_vnc\\results\\whole_brain_stats',
 'data_file': 'dff_1_5_5_with_ep_2020_08_27_12_32_48_526097.pkl',
 'cut_off_time': 3.656,
 'manip_type': 'both',
 'min_n_subjects_per_beh': 3,
 'test_type': 'decision_dependence',
 'beh_ref': 'Q',
 'alpha': 0.05,
 'save_folder': '\\\\dm11\\bishoplab\\projects\\keller_vnc\\results\\whole_brain_stats',
 'save_str': 'decision_dep_quiet_ref'}